In [5]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import os

import pg8000


In [20]:
#collect specimen name from user
sample = raw_input('Enter Specimen Name for Composite Image: ')
#capitalize first letter for search
sample = sample[0].upper() + sample[1:].strip()

Enter Specimen Name for Composite Image: Htr3a-Cre_NO152;Ai14-277453.04.02


In [21]:
conn = pg8000.connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432)
cur = conn.cursor()

cur.execute("select s.id, s.storage_directory from specimens s where s.name ilike %s", ('%' + sample,))
result = cur.fetchone()
if result is None:
    print "Could not find specimen result for " + specimen + ". Skipping..."
    

In [27]:
def LinuxtoWindow(Linuxpath):
    SplitLinux = Linuxpath.split('/')
    try:
        return (r"\\" + os.path.join(SplitLinux[1], SplitLinux[2],SplitLinux[3],SplitLinux[4], SplitLinux[5],SplitLinux[6],SplitLinux[7],''))
    except (WindowsError, AttributeError, IndexError):
        try:
            return (r"\\" + os.path.join('titan','cns', SplitLinux[2], SplitLinux[4],SplitLinux[5],''))
        except (WindowsError, AttributeError, IndexError):
            print 'Could not find cell'

In [28]:
LinuxtoWindow(result[1])

u'\\\\titan\\cns\\mousecelltypes\\prod828\\specimen_554013361\\'

In [ ]:
conn = pg8000.connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432)
    cur = conn.cursor()
    
    cur.execute("SELECT s.name, s.ephys_roi_result_id, s.id FROM specimens s WHERE s.name LIKE %s", ('%' + specimen,))
    result = cur.fetchone()
    if result is None:
        print "Could not find specimen result for " + specimen + ". Skipping..."
        return None
    specimen_name = result[0]
    ephys_roi_result_id = result[1]


    cur.execute("SELECT f.filename, f.storage_directory FROM well_known_files f \
                 WHERE f.attachable_type = 'EphysRoiResult' AND f.attachable_id = %s AND f.filename LIKE '%%ephys_features.json'", 
                 (ephys_roi_result_id,))
    result = cur.fetchone()
    if result is None:
        print "Could not find json file for " + specimen + ". Skipping..."
        return None

In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import os
#import allensdk_lims.internal.core.lims_utilities as lims_utilities

def LinuxtoWindow(Linuxpath):
    SplitLinux = Linuxpath.split('/')
    return (r"\\" + os.path.join(SplitLinux[1], SplitLinux[2],SplitLinux[3],SplitLinux[4], SplitLinux[5],SplitLinux[6],SplitLinux[7],''))

#collect specimen name from user
sample = raw_input('Enter Specimen Name for Composite Image: ')
#capitalize first letter for search
sample = sample[0].upper() + sample[1:].strip()

conn = pg8000.connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432)
cur = conn.cursor()
query = """
select s.id, s.storage_directory from specimens s
where s.name ilike '%s'
""" % ('%%' + sample )

result = lims_utilities.query(query)[0]

print result['storage_directory']

stor_dir_win = LinuxtoWindow(result['storage_directory'])
json_path = stor_dir_win + 'EPHYS_COMPOSITE_IMAGE_' + str(result['id']) + '_input.json'
png_path = stor_dir_win + 'composite_image.png'

os.startfile(png_path)

with open(json_path) as data_file:    
    data = json.load(data_file)

corner_df = pd.DataFrame(columns = ['Image Path','ID','Coord_x','Coord_y','Symbol','Color'])
j=0
for i in data['corners']:
    if 'failed' in i:
        corner_df.loc[j] = i,None,data['corners'][i]['x'],data['corners'][i]['y'],'D','r'
    else:
        corner_df.loc[j] = i,None,data['corners'][i]['x'],data['corners'][i]['y'],'s','k'
    j +=1
for i in data['recorded_cells']:
    corner_df.loc[j] = i,data['recorded_cells'][i]['name'][-3:],data['recorded_cells'][i]['x'],data['recorded_cells'][i]['y'],'o','g'
    j += 1

for i in range(len(corner_df)):
    plt.plot(corner_df['Coord_x'][i],corner_df['Coord_y'][i],marker = corner_df['Symbol'][i], c = corner_df['Color'][i])
    if corner_df['Symbol'][i] == 'o':
        plt.annotate(corner_df['ID'][i], xy=(corner_df['Coord_x'][i],corner_df['Coord_y'][i]))


plt.show()
plt.clf()


ImportError: No module named allensdk_lims.internal.core.lims_utilities

In [26]:
result

[554013361, u'/projects/mousecelltypes/vol1/prod828/specimen_554013361/']

In [25]:
def LinuxtoWindow(Linuxpath):
    SplitLinux = Linuxpath.split('/')
    return (r"\\" + os.path.join(SplitLinux[1], SplitLinux[2],SplitLinux[3],SplitLinux[4], SplitLinux[5],SplitLinux[6],SplitLinux[7],''))

LinuxtoWindow(result[1])

IndexError: list index out of range

In [7]:
result['storage_directory'].split('/')

[u'',
 u'allen',
 u'programs',
 u'celltypes',
 u'production',
 u'mousecelltypes',
 u'prod120',
 u'specimen_601790148',
 u'']

In [8]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import os
import pg8000

def LinuxtoWindow(Linuxpath):
    #sorts out the paths listed in LIMS
    
    SplitLinux = Linuxpath.split('/')
    if 'allen' in Linuxpath:
        return (r"\\" + os.path.join(SplitLinux[0], SplitLinux[1],SplitLinux[2], 'celltypes',SplitLinux[4],SplitLinux[5],SplitLinux[6],SplitLinux[7],''))
    else:
        return (r"\\" + os.path.join(SplitLinux[0],'allen','programs','celltypes', 'production',
                                     SplitLinux[2], SplitLinux[4],SplitLinux[5],''))

#collect specimen name from user
sample = raw_input('Enter Specimen Name for Composite Image: ')
#capitalize first letter for search
sample = sample[0].upper() + sample[1:].strip()


conn = pg8000.connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432)
cur = conn.cursor()

cur.execute("select s.id, s.storage_directory from specimens s where s.name ilike %s", ('%' + sample,))
result = cur.fetchone()
if result is None:
    print "Could not find specimen result for " + sample + ". Skipping..."

if result[1] is not None:
    
    
    stor_dir_win = LinuxtoWindow(result[1])
    json_path = stor_dir_win + 'EPHYS_COMPOSITE_IMAGE_' + str(result[0]) + '_input.json'
    png_path = stor_dir_win + 'composite_image.png'

    os.startfile(png_path)

    with open(json_path) as data_file:    
        data = json.load(data_file)

    corner_df = pd.DataFrame(columns = ['Image Path','ID','Coord_x','Coord_y','Symbol','Color'])
    j=0
    for i in data['corners']:
        if 'failed' in i:
            corner_df.loc[j] = i,None,data['corners'][i]['x'],data['corners'][i]['y'],'D','r'
        else:
            corner_df.loc[j] = i,None,data['corners'][i]['x'],data['corners'][i]['y'],'s','k'
        j +=1
    for i in data['recorded_cells']:
        corner_df.loc[j] = i,data['recorded_cells'][i]['name'][-3:],data['recorded_cells'][i]['x'],data['recorded_cells'][i]['y'],'o','g'
        j += 1

    for i in range(len(corner_df)):
        plt.plot(corner_df['Coord_x'][i],corner_df['Coord_y'][i],marker = corner_df['Symbol'][i], c = corner_df['Color'][i])
        if corner_df['Symbol'][i] == 'o':
            plt.annotate(corner_df['ID'][i], xy=(corner_df['Coord_x'][i],corner_df['Coord_y'][i]))


    plt.show()
    plt.clf()

elif result[1] is None:
    print "Could not find file path for this cell"
    

Enter Specimen Name for Composite Image: H17.26.003.11.15.02
Could not find file path for this cell


In [50]:
sample = 'Pvalb-IRES-Cre;Ai14-359337.04.02.01'

In [14]:
sample[0].upper() + sample[1:].strip()

'H17.26.003.11.15.02'

In [24]:
query = ("select cell.id, cell.storage_directory from ephys_roi_results where cell.name ilike %s")

In [61]:
conn = pg8000.connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432)
cur = conn.cursor()

cur.execute("SELECT s.name, s.ephys_roi_result_id, s.id FROM specimens s WHERE s.name iLIKE %s", ('%' + sample,))
result = cur.fetchone()

cur.execute("SELECT f.filename, f.storage_directory FROM well_known_files f \
                     WHERE f.attachable_type = 'EphysRoiResult' AND f.attachable_id = %s AND f.filename LIKE '%%ephys_features.json'", 
                     (650077353,))
results = cur.fetchone()


In [62]:
results

In [7]:
stor_dir_win = LinuxtoWindow('/projects/mousecelltypes/vol1/prod457/Ephys_Specimen_Roi_plan_484566220/')
#LinuxtoWindow(result[1])
json_path = stor_dir_win + 'EPHYS_COMPOSITE_IMAGE_' + str(result[0]) + '_input.json'
png_path = stor_dir_win + 'composite_image.png'

os.startfile(png_path)

with open(json_path) as data_file:    
    data = json.load(data_file)

corner_df = pd.DataFrame(columns = ['Image Path','ID','Coord_x','Coord_y','Symbol','Color'])
j=0
for i in data['corners']:
    if 'failed' in i:
        corner_df.loc[j] = i,None,data['corners'][i]['x'],data['corners'][i]['y'],'D','r'
    else:
        corner_df.loc[j] = i,None,data['corners'][i]['x'],data['corners'][i]['y'],'s','k'
    j +=1
for i in data['recorded_cells']:
    corner_df.loc[j] = i,data['recorded_cells'][i]['name'][-3:],data['recorded_cells'][i]['x'],data['recorded_cells'][i]['y'],'o','g'
    j += 1

for i in range(len(corner_df)):
    plt.plot(corner_df['Coord_x'][i],corner_df['Coord_y'][i],marker = corner_df['Symbol'][i], c = corner_df['Color'][i])
    if corner_df['Symbol'][i] == 'o':
        plt.annotate(corner_df['ID'][i], xy=(corner_df['Coord_x'][i],corner_df['Coord_y'][i]))


plt.show()
plt.clf()


WindowsError: [Error 2] The system cannot find the file specified: '\\\\allen\\programs\\celltypes\\production\\mousecelltypes\\prod457\\Ephys_Specimen_Roi_plan_484566220\\composite_image.png'

In [ ]:
st